## deltaLens Data Compare Sample

In [1]:
#imports

from delta_lens.deltaLens import DeltaLens, EntityComparer
from delta_lens.config import Config, Entity, Side, Transform, Defaults,ReferenceDataset, load_config
from delta_lens.sqliteExport import *
from delta_lens.csvExport import *
from datetime import date

In [2]:
#config

config = load_config("data/compare.config.json")


In [3]:
runName = f'tradeCompare_{date.today().strftime("%Y-%m-%d")}'

comparator = DeltaLens(runName,config)

comparator.execute(continue_on_error=True)



#### entity_compare_results
Table containing summary of each entity comparison

In [4]:
comparator.con.execute("SELECT * FROM  entity_compare_results").fetch_df()

,entity,rows_left,rows_right,rows_fully_matched,error_text,success
0,trade,42999117,43000000,42113850,None,1


#### [entitiy]_compare_field_summary
Entity specific table containing summary of field comparison

In [5]:
comparator.con.execute("SELECT * FROM trade_compare_field_summary").fetch_df()

,field,total,matches,match_percentage
0,trade_type,42999117,42999117.0,100.000000
1,symbol,42999117,42999117.0,100.000000
2,quantity,42999117,42999117.0,100.000000
3,timestamp,42999117,42999117.0,100.000000
4,price,42999117,42113850.0,97.940002


#### [entity]_compare
Entity specific table containing details of comparison

In [6]:
comparator.con.execute(" WITH CTE as (SELECT * FROM  trade_compare WHERE _full_match = 0) SELECT * FROM CTE USING SAMPLE 5;").fetch_df()

,trade_id,trade_type_left,trade_type_right,trade_type_match,symbol_left,symbol_right,symbol_match,quantity_left,quantity_right,quantity_match,timestamp_left,timestamp_right,timestamp_match,price_left,price_right,price_match,_exists_left,_exists_right,_full_match
0,2310919,BUY,BUY,True,MSFT,MSFT,True,180,180,True,2024-05-21 20:34:19,2024-05-21 20:34:19,True,786.57,179.74,False,True,True,False
1,14885155,BUY,BUY,True,TSLA,TSLA,True,5816,5816,True,2024-10-01 20:35:51,2024-10-01 20:35:51,True,941.70,831.15,False,True,True,False
2,29397447,SELL,SELL,True,NVDA,NVDA,True,5216,5216,True,2024-09-15 20:37:39,2024-09-15 20:37:39,True,735.65,506.85,False,True,True,False
3,29514608,SELL,SELL,True,WMT,WMT,True,6835,6835,True,2024-07-22 20:37:39,2024-07-22 20:37:39,True,646.86,572.69,False,True,True,False
4,38084072,SELL,SELL,True,MSFT,MSFT,True,1978,1978,True,2024-06-07 20:38:43,2024-06-07 20:38:43,True,57.16,340.83,False,True,True,False


#### Export to sqlite
Function to export above tables to on-disk format. Sqlite is a popular single file database, well supported in many languages,  for which lots of query tools exist. See [Datasette](https://datasette.io/) package.

In [7]:
export_to_sqlite(comparator.con, f'{runName}.sqlite')

FileExistsError: SQLite database file 'tradeCompare_2025-02-21.sqlite' already exists

In [ ]:
export_to_csv_archive(comparator.con,f'{runName}.tar.gz' )

'tradeCompare_2025-02-21.tar.gz'